In [ ]:
%cd ~/perpetual_day/

In [ ]:
import os
from pathlib import Path
from shutil import rmtree

import lightning as L
import pyearthtools.pipeline as petpipe
import matplotlib.pyplot as plt

from pipeline import filter_day_time, features_pipeline, target_pipeline
from datamodule import PetDataModule
from models import CNN

In [ ]:
date_range = petpipe.iterators.DateRange('20200101T0000', '20200101T2000', interval='30 minutes')
bbox = [-35, -25, 138, 150]
cache_dir = Path(os.environ["TMPDIR"]) / "pipeline_cache"

In [ ]:
valid_range = filter_day_time(date_range, bbox)
featpipe = features_pipeline(bbox)
targetpipe = target_pipeline(bbox)
fullpipe = petpipe.Pipeline((featpipe, targetpipe), iterator=valid_range)

Uncomment the following celle to remove the cache folder, forcing a preprocessing step.

In [ ]:
# rmtree(cache_dir)

In [ ]:
dm = PetDataModule(
    fullpipe,
    cache_dir=cache_dir,
    n_jobs=6,
    val_split=0.1,
    test_split=0.8,
    batch_size=16,
    num_workers=4,
    multiprocessing_context="forkserver",
    persistent_workers=True
)

In [ ]:
%%time
model = CNN(chan_in=9, chan_out=1)
trainer = L.Trainer(max_epochs=1, precision="16-mixed")
trainer.fit(model, dm)

In [ ]:
# TODO print results